In [1]:
import os
path = os.getcwd()

In [3]:
import warnings
warnings.filterwarnings("ignore")


In [1]:

import pandas as pd
import numpy as np
import ast
import re


import nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [4]:
movie = pd.read_csv(path + r'\trial data\tmdb_5000_movies.csv')
credit = pd.read_csv(path + r'\trial data\tmdb_5000_credits.csv')



In [5]:
movie.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [7]:
credit.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [8]:
root = pd.merge(movie , credit , on='title')

In [9]:
root.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [10]:
root.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'movie_id', 'cast', 'crew'],
      dtype='object')

In [11]:
'id' , 'title' , 'genres' , 'keywords' , 'overview' , 'cast' , 'crew'

('id', 'title', 'genres', 'keywords', 'overview', 'cast', 'crew')

In [12]:
data = root[['id' , 'title' , 'genres' , 'keywords' , 'overview' , 'cast' , 'crew']]

In [13]:
data.head(1)

,id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


### Pre-Processing

In [14]:
data.head(1)

,id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [15]:
data.isnull().sum()

id          0
title       0
genres      0
keywords    0
overview    3
cast        0
crew        0
dtype: int64

In [16]:
data.dropna(inplace=True)

In [17]:
data.isnull().sum()

id          0
title       0
genres      0
keywords    0
overview    0
cast        0
crew        0
dtype: int64

In [18]:
data.duplicated().sum()

0

#### Genres

In [19]:
data.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [20]:
def extract_name(items):
    list = []
    items = ast.literal_eval(items)
    for i in items:
        list.append(i['name'])
    return list

In [21]:
data['genres'] = data['genres'].apply(extract_name)

In [22]:
data['genres'][0]

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [23]:
data['genres'][0]

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

#### Keywords

In [24]:
data['keywords'][0]

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [25]:
data['keywords'] = data['keywords'].apply(extract_name)

#### Cast

##### Here we are taking top 3 actors, because top 3 actors are the important actors in every movies

In [26]:
# data['cast'][0]

In [27]:
def cast_name(items):
    list = []
    items = ast.literal_eval(items)
    index = 0
    for i in items:
        if index !=3:
            list.append(i['name'])
            index += 1
        else:
            break
    return list

In [28]:
data['cast'] = data['cast'].apply(cast_name)

In [29]:
data['cast'][0]

['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver']

#### Crew

##### Here we are taking only director, because all others are not important for recommender

In [30]:
# data['crew'][0]

In [31]:
def director_name(items):
    list = []
    items = ast.literal_eval(items)
    for i in items:
        if i['job'] == 'Director':
            list.append(i['name'])
            break
    return list

In [32]:
data['crew'] = data['crew'].apply(director_name)

In [33]:
data['crew'].tail(3)

4806         [Scott Smith]
4807         [Daniel Hsia]
4808    [Brian Herzlinger]
Name: crew, dtype: object

#### Making a copy of data for app 

In [34]:
data_app = data[['id','title','genres','overview','cast','crew']]

#### Overview

In [35]:
data['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

In [36]:
data['overview'] = data['overview'].apply(lambda i: i.split())

#### Removing space from cast, crew, keywored and genres

In [37]:
data['genres'] = data['genres'].apply(lambda i : [j.replace(' ','') for j in i])

In [38]:
data['keywords'] = data['keywords'].apply(lambda i : [j.replace(' ','') for j in i])

In [39]:
data['cast'] = data['cast'].apply(lambda i : [j.replace(' ','') for j in i])

In [40]:
data['crew'] = data['crew'].apply(lambda i : [j.replace(' ','') for j in i])

#### Making a new column tokens

In [41]:
data['tokens'] = data['cast'] + data['crew'] + data['genres'] + data['keywords'] + data['overview']

In [42]:
data.head(1)

,id,title,genres,keywords,overview,cast,crew,tokens
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[In, the, 22nd, century,, a, paraplegic, Marin...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[SamWorthington, ZoeSaldana, SigourneyWeaver, ..."


#### Making new Data-Frame

In [43]:
new_data = data[['id' , 'title' , 'tokens']]

In [44]:
new_data.head()

,id,title,tokens
0,19995,Avatar,"[SamWorthington, ZoeSaldana, SigourneyWeaver, ..."
1,285,Pirates of the Caribbean: At World's End,"[JohnnyDepp, OrlandoBloom, KeiraKnightley, Gor..."
2,206647,Spectre,"[DanielCraig, ChristophWaltz, LéaSeydoux, SamM..."
3,49026,The Dark Knight Rises,"[ChristianBale, MichaelCaine, GaryOldman, Chri..."
4,49529,John Carter,"[TaylorKitsch, LynnCollins, SamanthaMorton, An..."


In [45]:
# new_data['tokens'][0]

In [46]:
new_data['tokens'] = new_data['tokens'].apply(lambda i : ' '.join(i))

In [47]:
new_data['tokens'] = new_data['tokens'].apply(lambda i : i.lower())

In [48]:
new_data.head()

,id,title,tokens
0,19995,Avatar,samworthington zoesaldana sigourneyweaver jame...
1,285,Pirates of the Caribbean: At World's End,johnnydepp orlandobloom keiraknightley gorever...
2,206647,Spectre,danielcraig christophwaltz léaseydoux sammende...
3,49026,The Dark Knight Rises,christianbale michaelcaine garyoldman christop...
4,49529,John Carter,taylorkitsch lynncollins samanthamorton andrew...


#### Cleansing

In [49]:
def clean_text(token):
    
    token = re.sub(r'\[[0-9]*\]', ' ',token)
    token = re.sub(r'\s+', ' ', token)
    token = re.sub('[^a-zA-Z]', ' ', token )
    token = re.sub(r'\s+', ' ', token)
    
    return token

In [50]:
new_data['tokens'] = new_data['tokens'].apply(clean_text)

In [51]:
word = word_tokenize
stemmer = PorterStemmer()

In [52]:
def stem(token):
    token = word(token)
    token = [stemmer.stem(x) for x in token]
    return ' '.join(token)

In [53]:
new_data['tokens'] = new_data['tokens'].apply(stem)

#### Vectorization

In [54]:
new_data['tokens'][0]

'samworthington zoesaldana sigourneyweav jamescameron action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul d in the nd centuri a parapleg marin is dispatch to the moon pandora on a uniqu mission but becom torn between follow order and protect an alien civil'

In [55]:
tfidf = TfidfVectorizer(max_features=5000 , stop_words='english')

In [56]:
vectors = tfidf.fit_transform(new_data['tokens']).toarray()

In [57]:
# list(tfidf.get_feature_names_out())

In [58]:
vectors

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

#### Finding Similarity

##### Here we are using Cosine Similarity instead of Euclidean Distance.

In [59]:
similarity = cosine_similarity(vectors)

In [60]:
sorted(enumerate(similarity[0]) , reverse=True , key= lambda x : x[1])

[(0, 1.0),
 (2409, 0.2686399725888631),
 (3730, 0.19979624522845663),
 (582, 0.17218768130870638),
 (3608, 0.1637100584828132),
 (778, 0.1554159824453532),
 (1204, 0.15086525561125635),
 (539, 0.14844079993530182),
 (260, 0.14640644875261327),
 (1920, 0.14490284985777993),
 (942, 0.14401855391898977),
 (47, 0.14245312410726896),
 (838, 0.1399002113891752),
 (1216, 0.13713505732354234),
 (557, 0.13354393470184212),
 (507, 0.1332505890480841),
 (438, 0.12789086681749368),
 (3538, 0.12782260654493238),
 (1344, 0.12631795778574917),
 (311, 0.12580870515614762),
 (1089, 0.12460471688064093),
 (2786, 0.12315445721735753),
 (2284, 0.12263670472167659),
 (83, 0.12245309541831703),
 (3736, 0.12132102089120288),
 (2938, 0.12118163885670877),
 (74, 0.12082630520404496),
 (3218, 0.12017131750225378),
 (4336, 0.11846740799483274),
 (2204, 0.11786261663703262),
 (1831, 0.11631748639199836),
 (1747, 0.11615675135299178),
 (4048, 0.1158769248587853),
 (300, 0.11402060150839799),
 (1348, 0.113367687750

In [61]:
new_data.iloc[582].title

'Battle: Los Angeles'

In [62]:
new_data[new_data['title'] == 'Battle: Los Angeles'].index[0]

582

#### Recommendation System

In [63]:
def recommender(name):

    index = new_data[new_data['title'] == name].index[0]
    top6 = sorted(enumerate(similarity[index]) , reverse=True , key= lambda x : x[1])[1:7]
    
    for i in top6:
        movie = new_data.iloc[i[0]].title
        print(movie)
        

In [64]:
recommender('Avatar')

Aliens
Falcon Rising
Battle: Los Angeles
Apollo 18
Meet Dave
Predators


In [65]:
recommender('Battle: Los Angeles')

Sands of Iwo Jima
Falcon Rising
Avatar
Skyline
Force 10 from Navarone
The Thin Red Line


In [66]:
def movie(name):
    
    name = name.lower()
    new_title = [re.sub(r'[^\w\s]', '', title.lower()) for title in new_data['title']]

    if name in new_title:
        index = new_title.index(name)
        recommender(new_data.iloc[index].title)
    else:
        print(f'No movie available for "{name}"')

In [67]:
# name = input('Enter Movie Name : ')
# movie(name)

#### Generates Pickle files for app

In [68]:
data_app.head(1)

,id,title,genres,overview,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","In the 22nd century, a paraplegic Marine is di...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [69]:
data_dict = data_app.to_dict()

In [70]:
pickle.dump(data_dict, open('deployment/movie_dict.pkl', 'wb'))

In [71]:
pickle.dump(similarity, open('deployment/similarity.pkl', 'wb'))